# Simulation experiment 

Run entire simulation experiment multiple times to generate confidence interval

In [1]:
%load_ext autoreload
%autoreload 2

from joblib import Parallel, delayed
import multiprocessing
import sys
import os
import pandas as pd

import warnings
warnings.filterwarnings(action='ignore')

sys.path.append("../")
from functions import pipelines

from numpy.random import seed
randomState = 123
seed(randomState)

Using TensorFlow backend.


In [2]:
# Parameters
NN_architecture = 'NN_2500_30'
analysis_name = 'analysis_2'
file_prefix = 'Experiment_corrected'
num_simulated_samples = 500
lst_num_experiments = [1, 2, 5, 10, 20,
                     50, 100, 250, 500]
corrected = True
use_pca = True
num_PCs = 10

iterations = range(5) 
num_cores = 5

In [3]:
# Input
local_dir = "/home/alexandra/Documents/"

normalized_data_file = os.path.join(
  local_dir,
  "Data",
  "Batch_effects",
  "input",
  "recount2_gene_normalized_data.tsv")

In [4]:
# Output files
local_dir = "/home/alexandra/Documents/"

similarity_corrected_file = os.path.join(
    local_dir,
    "Data",
    "Batch_effects",
    "output",
    "saved variables",
    "analysis_2_similarity_corrected.pickle")

ci_corrected_file = os.path.join(
    local_dir,
    "Data",
    "Batch_effects",
    "output",
    "saved variables",
    "analysis_2_ci_corrected.pickle")

In [5]:
# Run multiple simulations - corrected
results = Parallel(n_jobs=num_cores, verbose=100)(
    delayed(
        pipelines.simple_simulation_experiment_corrected)(i,
                                                          NN_architecture,
                                                          analysis_name,
                                                          num_simulated_samples,
                                                          lst_num_experiments,
                                                          corrected,
                                                          use_pca,
                                                          num_PCs,
                                                          "Experiment_corrected",
                                                          normalized_data_file) for i in iterations)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   1 tasks      | elapsed: 79.7min
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed: 79.7min remaining: 119.5min
[Parallel(n_jobs=5)]: Done   3 out of   5 | elapsed: 79.8min remaining: 53.2min
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 79.9min remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 79.9min finished


In [6]:
# Concatenate output dataframes
all_svcca_scores = pd.DataFrame()

for i in iterations:
    all_svcca_scores = pd.concat([all_svcca_scores, results[i][1]], axis=1)

all_svcca_scores

,score,score,score,score,score
number of experiments,,,,,
1,0.999987,0.999989,0.999987,0.999988,0.999988
2,0.998200,0.997626,0.998684,0.999167,0.999157
5,0.995581,0.994608,0.995169,0.996330,0.995033
10,0.989566,0.992462,0.992442,0.991100,0.990315
20,0.980842,0.982218,0.980210,0.981299,0.980843
50,0.949096,0.956317,0.950820,0.946067,0.950263
100,0.898952,0.892437,0.898010,0.894421,0.892205
250,0.691763,0.706856,0.698909,0.725621,0.704418
500,0.113064,0.110935,0.115193,0.120960,0.125141


In [7]:
# Get median for each row (number of experiments)
mean_scores = all_svcca_scores.mean(axis=1).to_frame()
mean_scores.columns = ['score']
mean_scores

,score
number of experiments,
1,0.999988
2,0.998567
5,0.995344
10,0.991177
20,0.981083
50,0.950513
100,0.895205
250,0.705513
500,0.117059


In [8]:
# Get standard dev for each row (number of experiments)
import math
std_scores = (all_svcca_scores.std(axis=1)/math.sqrt(10)).to_frame()
std_scores.columns = ['score']
std_scores

,score
number of experiments,
1,2.590993e-07
2,2.087016e-04
5,2.060063e-04
10,4.060081e-04
20,2.351610e-04
50,1.179090e-03
100,9.896770e-04
250,3.999927e-03
500,1.853983e-03


In [9]:
# Get confidence interval for each row (number of experiments)
# z-score for 95% confidence interval
err = std_scores*1.96

In [10]:
# Get boundaries of confidence interval
ymax = mean_scores + err
ymin = mean_scores - err

ci = pd.concat([ymin, ymax], axis=1)
ci.columns = ['ymin', 'ymax']
ci

,ymin,ymax
number of experiments,,
1,0.999987,0.999988
2,0.998158,0.998976
5,0.994940,0.995748
10,0.990381,0.991973
20,0.980622,0.981543
50,0.948202,0.952824
100,0.893265,0.897145
250,0.697673,0.713353
500,0.113425,0.120692


In [11]:
mean_scores

,score
number of experiments,
1,0.999988
2,0.998567
5,0.995344
10,0.991177
20,0.981083
50,0.950513
100,0.895205
250,0.705513
500,0.117059


In [12]:
# Pickle dataframe of mean scores scores for first run, interval
mean_scores.to_pickle(similarity_corrected_file)
ci.to_pickle(ci_corrected_file)